In [ ]:
#Import Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
#import dataset 1 Hotels / campings / autres
nr_import_a = pd.read_excel("20241024-Export-hotels--campings--aires-et-residences.xlsx")

#drop specific columns to adhere to import_2 column structure
nr_import_a = nr_import_a.drop(columns=["Nombre de chambres déclarées", "Nombre d'appartement", "Emplacements nus", "Mobil-Home" , "Bungalow","Chalet","Emplacements déclarés" , "Emplacements Locatifs", "Emplacements résidentiels"])

In [3]:
#import dataset 2 locations
nr_import_b = pd.read_excel("20241024-Export-locations.xlsx")

In [4]:
# Concatenate nr_import_a and nr_import_b vertically
nr_import = pd.concat([nr_import_a, nr_import_b], axis=0, ignore_index=True)

In [5]:
#removing rows where "nom de l'offre" is NaN and storing in seperate file

# Separate rows with NaN in "Nom de l'offre" into a new DataFrame
nr_name_nan = nr_import[nr_import["Nom de l'offre"].isna()]

# Drop rows with NaN in "Nom de l'offre" from the original DataFrame
nr_import_2 = nr_import.dropna(subset=["Nom de l'offre"])

# Save the separate DataFrame with missing "Nom de l'offre" to an Excel file
nr_name_nan.to_excel("nr_name_nan.xlsx", index=False)

In [6]:
#Flagging potential duplicates in original df and create an export file for duplicates review

# Create nr_import_3 as a copy of nr_import_2
nr_import_3 = nr_import_2.copy()

# Step 1: Mark potential duplicates in a new column "duplicate" in nr_import_3
nr_import_3['Duplicate'] = nr_import_3.duplicated(subset=["Nom de l'offre", "Adresse"], keep='first')

# Step 2: Create a DataFrame containing all duplicates, including the first occurrence, for review
nr_duplicates_for_review = nr_import_3[nr_import_3['Duplicate'] | nr_import_3.duplicated(subset=["Nom de l'offre", "Adresse"], keep=False)]

# Sort duplicates_for_review by "Nom de l'offre" in ascending order
nr_duplicates_for_review = nr_duplicates_for_review.sort_values(by="Nom de l'offre")

# Export the duplicated lines (including first occurrences) to an Excel file
nr_duplicates_for_review.to_excel("nr_duplicated_rows_review.xlsx", index=False)

In [7]:
#dropping & renaming columns / data casting

#dropping  
nr_import_4 = nr_import_3.drop(columns=["Label Tourisme et handicap", "Modes de paiement acceptés", "Groupes acceptés", "Types de groupes acceptés","Tarifs"])

#renaming columns
nr_import_5 = nr_import_4.rename(columns={"Nom de l'offre": "Offre", "Code postal" : "Code_postal", "Animaux acceptés":"Animaux_acceptes", "Catégorie":"Categorie", "Capacité totale":"Capacite","Services et équipements":"Services"})

#Data type casting Code_postal & INSEE to object
# First, cast to integer to remove the decimal, then to string for final output
nr_import_5["Code_postal"] = nr_import_5["Code_postal"].fillna(0).astype(int).astype(str)
nr_import_5["INSEE"] = nr_import_5["INSEE"].fillna(0).astype(int).astype(str)

In [8]:
#treating label column / extracting list / create column label_count

# Replace "Vélo" with "vélo" in the 'Label' column
nr_import_5['Label'] = nr_import_5['Label'].str.replace(r'\bVélo\b', 'vélo', regex=True)

# Drop NaN values in the "Label" column and ensure all values are treated as strings
all_labels = nr_import_5['Label'].dropna().astype(str)

# Split each cell by commas, expand the results, strip spaces, and get unique values
all_labels = all_labels.str.split(',').explode().str.strip()

# Get unique labels, converting to a sorted list
unique_labels = sorted(all_labels.unique())

# Create a DataFrame from the sorted list and add an index column
labels_list_db = pd.DataFrame(unique_labels, columns=["Label"]).sort_values(by="Label").reset_index()
labels_list_db.rename(columns={"index": "Label_ID"}, inplace=True)  # Renaming the index column if desired

# Export to Excel, with the Label_ID as a column in the sheet
labels_list_db.to_excel("labels_list_db.xlsx", index=False)

In [9]:
# Create a new column "label_count" that counts the number of different labels in the "Label" column
# Split each entry by commas, count the parts, and handle NaN values by filling with 0
nr_import_5['Label_count'] = nr_import_5['Label'].fillna('').apply(lambda x: len(x.split(',')) if x else 0)


In [10]:
#identifying opening scheme with regex

import re

# Create nr_import_6 as a copy of nr_import_5
nr_import_6 = nr_import_5.copy()

# Define the function to determine the opening scheme
def determine_opening_scheme(row):
    # Step 1: Check "Période ouverture et fermeture" columns first
    periode_text = " ".join(str(row[col]) for col in [
        "Période ouverture et fermeture 1", "Période ouverture et fermeture 2", "Période ouverture et fermeture 3"
    ] if pd.notna(row[col])).strip()

    # Check for all-year patterns in "Période ouverture et fermeture" columns
    if re.search(r"toute l'année|all year|année complète|year round", periode_text, re.IGNORECASE):
        return "Annee"
    elif re.search(r"janvier|février|mars|avril|mai|juin|juillet|août|septembre|octobre|novembre|décembre|hiver|été|printemps|automne|saison", periode_text, re.IGNORECASE):
        return "Saison"
    
    # Step 2: If no result from "Période ouverture et fermeture" columns, check "Ouverture accueil (texte)"
    accueil_text = str(row["Ouverture accueil (texte)"]) if pd.notna(row["Ouverture accueil (texte)"]) else ""

    if re.search(r"toute l'année|all year|année complète|year round", accueil_text, re.IGNORECASE):
        return "Annee"
    elif re.search(r"janvier|février|mars|avril|mai|juin|juillet|août|septembre|octobre|novembre|décembre|hiver|été|printemps|automne|saison", accueil_text, re.IGNORECASE):
        return "Saison"
    
    # Default to blank if no pattern is detected
    return ""

# Apply the function to each row to create the "Ouverture" column in nr_import_6
nr_import_6['Ouverture'] = nr_import_6.apply(determine_opening_scheme, axis=1)

# Drop the columns that are no longer needed
nr_import_6 = nr_import_6.drop(columns=["Ouverture accueil (texte)", "Période ouverture et fermeture 1", "Période ouverture et fermeture 2","Période ouverture et fermeture 3"])


In [11]:
# Step 1: Extract unique values from the 'Type' column
unique_types = nr_import_6['Type'].unique()

# Step 2: Convert the unique values into a DataFrame
type_offre_db = pd.DataFrame(unique_types, columns=['Type_offre'])

# Step 3: Sort values alphabetically, treating blanks as '0' and assigning an index
type_offre_db = type_offre_db.sort_values(by='Type_offre', key=lambda x: x.replace('', '0')).reset_index(drop=True)

# Step 4: Assign a custom index, with blank types as index 0
type_offre_db['type_offre_ID'] = range(1, len(type_offre_db) + 1)
type_offre_db.loc[type_offre_db['Type_offre'] == '', 'type_offre_ID'] = 0

# Step 5: Reorder columns so 'Index' is the first column and sort by 'Index'
type_offre_db = type_offre_db[['type_offre_ID', 'Type_offre']].sort_values(by='type_offre_ID').reset_index(drop=True)

#export to XL
type_offre_db.to_excel("type_offre_db.xlsx", index=False)

In [12]:
# Check for duplicates in the TSID column
duplicate_tsid = nr_import_6['TSID'].duplicated().any()

if duplicate_tsid:
    print("There are duplicate TSID values.")
else:
    print("All TSID values are unique.")

All TSID values are unique.


In [16]:
# Step 1: Create a copy of nr_import_6
nr_import_7 = nr_import_6.copy()

# Step 2: Merge nr_import_7 with type_offre on the 'Type' column to get the corresponding IDs
nr_import_7 = nr_import_7.merge(type_offre_db, left_on='Type', right_on='Type_offre', how='left')

# Step 3: Rename the 'Index' column from type_offre to 'Type_ID' for clarity
nr_import_7 = nr_import_7.rename(columns={'Index': 'Type_offre_ID'})

# Step 4: Drop the 'Type_offre' column as it's no longer needed after the merge
nr_import_7 = nr_import_7.drop(columns=['Type_offre'])

#export to XL
nr_import_7.to_excel("backup_offer_all.xlsx", index=False)

In [17]:
#create a sub dataset with Service and description for future NLP

# Select the desired columns from nr_import_7 and create descriptis_db
nr_descriptif = nr_import_7[['TSID', 'Offre', 'Services', 'Descriptif']].copy()

#export to XL
nr_descriptif.to_excel("descriptif_db.xlsx", index=False)

In [19]:
# Create a copy of nr_import_7 and drop the specified columns
offres_normandie_db = nr_import_7.drop(columns=["Type", "Descriptif"]).copy()

#export to XL
offres_normandie_db.to_excel("offres_normandie_db.xlsx", index=False)
